[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/diogoflim/PO_II/blob/main/4_Transportes/demanda_fantasma.ipynb)

## **Pesquisa Operacional II**

**Prof. Diogo Ferreira de Lima Silva (TEP-UFF)**

## Modelando um Problema dos Transportes com capacidade ociosa

In [ ]:
#Execute esse bloco caso esteja executando no Google Colab
!pip install -q pyomo
!pip install -i https://pypi.gurobi.com gurobipy

In [ ]:
import pandas as pd
import numpy as np
import pyomo.environ as pyo

In [ ]:
origens = ['mes_1', 'mes_2', 'mes_3','mes_4']      # Nome das fábricas.
destinos = ['mes_1', 'mes_2', 'mes_3','mes_4'] # Nome dos centros de distribuição.

custos_dataframe = pd.DataFrame ([ [1080, 1095, 1110, 1125], # F1
                                    [9999999, 1110, 1125, 1140], # F2
                                    [9999999, 9999999, 1100, 1115],
                                    [9999999, 9999999, 9999999, 1130]], index = origens, columns= destinos)

custos_dataframe

In [ ]:
ofertas = [25, 35, 30, 10]
demandas = [10, 15, 25, 20]
custos = np.array(custos_dataframe) # Passando os custos para uma matriz do numpy

In [ ]:
modelo = pyo.ConcreteModel() # Criando uma instância do modelo

# Criando dois índices para serem usados no pyomo
I = modelo.I = pyo.RangeSet(len(ofertas)) # Índice para as fábricas
J = modelo.J = pyo.RangeSet(len(demandas)) # Índice para os centros de distribuição

# VARIÁVEIS DE DECISÃO
x = modelo.x = pyo.Var(I, J, within= pyo.NonNegativeReals)

# Custos de transporte da fábrica i para o centro de dsitribuição j
c = modelo.c = pyo.Param(I, J, initialize = lambda modelo, i, j: custos[i-1, j-1])

# Capacidade de cada fábrica
a = modelo.a = pyo.Param (I, initialize = lambda modelo, i: ofertas[i-1])

# Demanda de cada centro de distribuição
b = modelo.b = pyo.Param (J, initialize = lambda modelo, j: demandas[j-1])

# FUNÇÃO OBJETIVO
z = modelo.z = pyo.Objective(rule= lambda modelo: sum(x[i,j] * c[i,j] for i in I for j in J), sense= pyo.minimize)

# RESTRIÇÕES
# R1: O total de remessas despachada por cada fábrica não pode ser maior que a sua oferta.
modelo.R1 = pyo.Constraint(I, rule= lambda modelo, i: sum(x[i, j] for j in J) <= a[i])

# R2: Cada centro de distribuição deve ter a sua demanda atendida
modelo.R2 = pyo.Constraint(J, rule= lambda modelo, j: sum(x[i, j] for i in I) >= b[j])

# RESOLUÇÃO DO MODELO
gurobi = pyo.SolverFactory('gurobi') # Construindo o solver gurobi
resultado = gurobi.solve(modelo) # Armazenando o resultado

In [ ]:
pyo.value(z)

In [ ]:
for k in x.keys(): print (f"x_{k} = {pyo.value(x[k])}")

Perceba que chegamos a um ótimo, mesmo com uma demanda total diferente da oferta total! 

Isso ocorre porque conseguimos atender as demandas (temos capacidade ociosa). Em nossa restrição de demanda usamos (>=) e não obrigamos a igualdade (==).

Vamos adicionar restrições de igualdade para a oferta e para a demanda. Assim, não encontraremos uma solução viável! Não temos capacidade para isso.

In [ ]:
modelo = pyo.ConcreteModel() # Criando uma instância do modelo

# Criando dois índices para serem usados no pyomo
I = modelo.I = pyo.RangeSet(len(ofertas)) # Índice para as fábricas
J = modelo.J = pyo.RangeSet(len(demandas)) # Índice para os centros de distribuição

# VARIÁVEIS DE DECISÃO
x = modelo.x = pyo.Var(I, J, within= pyo.NonNegativeReals)

# Custos de transporte da fábrica i para o centro de dsitribuição j
c = modelo.c = pyo.Param(I, J, initialize = lambda modelo, i, j: custos[i-1, j-1])

# Capacidade de cada fábrica
a = modelo.a = pyo.Param (I, initialize = lambda modelo, i: ofertas[i-1])

# Demanda de cada centro de distribuição
b = modelo.b = pyo.Param (J, initialize = lambda modelo, j: demandas[j-1])

# FUNÇÃO OBJETIVO
z = modelo.z = pyo.Objective(rule= lambda modelo: sum(x[i,j] * c[i,j] for i in I for j in J), sense= pyo.minimize)

# RESTRIÇÕES
# R1: O total de remessas despachada por cada fábrica não pode ser maior que a sua oferta.
modelo.R1 = pyo.Constraint(I, rule= lambda modelo, i: sum(x[i, j] for j in J) == a[i])

# R2: Cada centro de distribuição deve ter a sua demanda atendida
modelo.R2 = pyo.Constraint(J, rule= lambda modelo, j: sum(x[i, j] for i in I) == b[j])

# RESOLUÇÃO DO MODELO
gurobi = pyo.SolverFactory('gurobi') # Construindo o solver gurobi
resultado = gurobi.solve(modelo) # Armazenando o resultado

In [ ]:
pyo.value(z)

In [ ]:
for k in x.keys(): print (f"x_{k} = {pyo.value(x[k])}")

### Incluindo uma Demanda Fantasma

In [ ]:
origens = ['mes_1', 'mes_2', 'mes_3','mes_4']      # Nome das fábricas.
destinos = ['mes_1', 'mes_2', 'mes_3','mes_4', 'fantasma'] # Nome dos centros de distribuição.

custos_dataframe = pd.DataFrame ([ [1080, 1095, 1110, 1125, 0], 
                                    [9999999, 1110, 1125, 1140, 0],
                                    [9999999, 9999999, 1100, 1115, 0],
                                    [9999999, 9999999, 9999999, 1130, 0]], index = origens, columns= destinos)

custos_dataframe

In [ ]:
ofertas = [25, 35, 30, 10]
demandas = [10, 15, 25, 20, 30]
custos = np.array(custos_dataframe) # Passando os custos para uma matriz do numpy

In [ ]:
modelo = pyo.ConcreteModel() # Criando uma instância do modelo

# Criando dois índices para serem usados no pyomo
I = modelo.I = pyo.RangeSet(len(ofertas)) # Índice para as fábricas
J = modelo.J = pyo.RangeSet(len(demandas)) # Índice para os centros de distribuição

# VARIÁVEIS DE DECISÃO
x = modelo.x = pyo.Var(I, J, within= pyo.NonNegativeReals)

# Custos de transporte da fábrica i para o centro de dsitribuição j
c = modelo.c = pyo.Param(I, J, initialize = lambda modelo, i, j: custos[i-1, j-1])

# Capacidade de cada fábrica
a = modelo.a = pyo.Param (I, initialize = lambda modelo, i: ofertas[i-1])

# Demanda de cada centro de distribuição
b = modelo.b = pyo.Param (J, initialize = lambda modelo, j: demandas[j-1])

# FUNÇÃO OBJETIVO
z = modelo.z = pyo.Objective(rule= lambda modelo: sum(x[i,j] * c[i,j] for i in I for j in J), sense= pyo.minimize)

# RESTRIÇÕES
# R1: O total de remessas despachada por cada fábrica não pode ser maior que a sua oferta.
modelo.R1 = pyo.Constraint(I, rule= lambda modelo, i: sum(x[i, j] for j in J) == a[i])

# R2: Cada centro de distribuição deve ter a sua demanda atendida
modelo.R2 = pyo.Constraint(J, rule= lambda modelo, j: sum(x[i, j] for i in I) == b[j])

# RESOLUÇÃO DO MODELO
gurobi = pyo.SolverFactory('gurobi') # Construindo o solver gurobi
resultado = gurobi.solve(modelo) # Armazenando o resultado

In [ ]:
print(f"z= {pyo.value(z)}")
for k in x.keys(): print (f"x_{k} = {pyo.value(x[k])}")